# pyspark

In [1]:
import pyspark
from pyspark import SparkContext
sc =SparkContext()

/home/souro/AI-ML-Training_Mindmajix/venv_aimltrn/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
nums = sc.parallelize([1,2,3,4])

In [3]:
nums.take(1)

[1]

In [4]:
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print('%i ' % (num))

1 
4 
9 
16 


In [5]:
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

/home/souro/AI-ML-Training_Mindmajix/venv_aimltrn/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
list_p = [('John',19),('Smith',29),('Adam',35),('Henry',50)]
rdd = sc.parallelize(list_p)
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))

In [8]:
DF_ppl = sqlContext.createDataFrame(ppl)
DF_ppl.printSchema()
DF_ppl.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-----+---+
| name|age|
+-----+---+
| John| 19|
|Smith| 29|
| Adam| 35|
|Henry| 50|
+-----+---+



In [9]:
from pyspark import SparkFiles
df = sqlContext.read.csv(SparkFiles.get("/home/souro/AI-ML-Training_Mindmajix/adult_data.csv"), header=True, inferSchema= True)

In [10]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [11]:
df.show(5, truncate = False)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|x  |age|workclass|fnlwgt|education   |educational-num|marital-status    |occupation       |relationship|race |gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|1  |25 |Private  |226802|11th        |7              |Never-married     |Machine-op-inspct|Own-child   |Black|Male  |0           |0           |40            |United-States |<=50K |
|2  |38 |Private  |89814 |HS-grad     |9              |Married-civ-spouse|Farming-fishing  |Husband     |White|Male  |0           |0           |50            |United-States |<=50K |
|3  |28 |Local-gov|336951|Assoc-acdm  |12             |Married-civ-spouse|Protective-serv 

In [13]:
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']
# Convert the type
df = convertColumn(df, CONTI_FEATURES, FloatType())
# Check the dataset
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [14]:
df.select('age','fnlwgt').show(5)

+----+--------+
| age|  fnlwgt|
+----+--------+
|25.0|226802.0|
|38.0| 89814.0|
|28.0|336951.0|
|44.0|160323.0|
|18.0|103497.0|
+----+--------+
only showing top 5 rows



In [15]:
df.groupBy("education").count().sort("count",ascending=True).show()

+------------+-----+
|   education|count|
+------------+-----+
|   Preschool|   83|
|     1st-4th|  247|
|     5th-6th|  509|
|   Doctorate|  594|
|        12th|  657|
|         9th|  756|
| Prof-school|  834|
|     7th-8th|  955|
|        10th| 1389|
|  Assoc-acdm| 1601|
|        11th| 1812|
|   Assoc-voc| 2061|
|     Masters| 2657|
|   Bachelors| 8025|
|Some-college|10878|
|     HS-grad|15784|
+------------+-----+



In [16]:
df.describe().show()

+-------+------------------+-----------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|summary|                 x|              age|  workclass|            fnlwgt|   education|   educational-num|marital-status|      occupation|relationship|              race|gender|      capital-gain|      capital-loss|    hours-per-week|native-country|income|
+-------+------------------+-----------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|  count|             48842|            48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|             48842|  

In [17]:
df.describe('capital-gain').show()

+-------+------------------+
|summary|      capital-gain|
+-------+------------------+
|  count|             48842|
|   mean|1079.0676262233324|
| stddev| 7452.019057655418|
|    min|               0.0|
|    max|           99999.0|
+-------+------------------+



In [18]:
df.filter(df.age > 40).count()

20211

# pymongo

In [ ]:
from pymongo import MongoClient

#Creating a pymongo client
client = MongoClient('localhost', 27017)

#Getting the database instance
db = client['mydb']
print("Database created........")

#Verification
print("List of databases after creating new one")
print(client.list_database_names())

In [ ]:
from pymongo import MongoClient

#Creating a pymongo client
client = MongoClient('localhost', 27017)

#Getting the database instance
db = client['mydb']

#Creating a collection
collection = db['souro_collection']
print("Collection created........")

In [ ]:
from pymongo import MongoClient

#Creating a pymongo client
client = MongoClient("mongodb://localhost:27017/")

#Getting the database instance
db = client['mydb']

#Creating a collection
coll = db['souro_collection']

#Inserting document into a collection
doc1 = {"name": "Souro", "id": "5", "city": "Prague"}
coll.insert_one(doc1)
print(coll.find_one())

In [ ]:
from pymongo import MongoClient

#Creating a pymongo client
client = MongoClient('localhost', 27017)

#Getting the database instance
db = client['mydb']

#Creating a collection
coll = db['souro_collection']

#Inserting document into a collection
data = [
   {
      "_id": "101", 
      "name": "Souro1", 
      "id": "1", 
      "city": "Prague1"
   },
   {
      "_id": "102", 
      "name": "Souro2", 
      "id": "2", 
      "city": "Prague2"
   },
   {
      "_id": "103", 
      "name": "Souro3", 
      "id": "3", 
      "city": "Prague3"
   }
]
res = coll.insert_many(data)
print("Data inserted ......")
print(res.inserted_ids)

In [ ]:
#Retrieving the first record using the find_one() method
print("First record of the collection: ")
print(coll.find_one())

#Retrieving a record with is 103 using the find_one() method
print("Record whose id is 103: ")
print(coll.find_one({"_id": "103"}))

In [ ]:
#Retrieving all the records using the find() method
print("Records of the collection: ")
for doc1 in coll.find():
    print(doc1)

In [ ]:
for doc1 in coll.find().sort("city"):
   print(doc1)

In [ ]:
coll.delete_one({"id" : "2"})

In [ ]:
collections = db.list_collection_names()
for coll in collections:
   print(coll)


In [ ]:
db.drop_collection(coll)